In [1]:
from pybit import inverse_perpetual  # <-- import HTTP & WSS for inverse perp
from pybit import spot  # <-- import HTTP & WSS for spot
from pybit import HTTP  # supports inverse perp & futures, usdt perp, spot.
from dotenv import load_dotenv
import os
import pandas as pd
import time
import ta 
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [2]:
load_dotenv()

True

In [2]:
#Loading my Bybit's API keys from the dotenv file
api_key_pw = os.getenv('api_key_bot')
api_secret_pw = os.getenv('api_secret_bot')


In [3]:
#Stablishing Connection
session = inverse_perpetual.HTTP(
    endpoint='https://api.bybit.com', 
    api_key= api_key_pw,
    api_secret= api_secret_pw
)
ws = inverse_perpetual.WebSocket(
    test=False,
    api_key= api_key_pw,
    api_secret= api_secret_pw
)

In [4]:
#Trying stuff to see how the API works
pd.DataFrame(session.latest_information_for_symbol(symbol ='BTCUSD')["result"]).keys()

Index(['symbol', 'bid_price', 'ask_price', 'last_price', 'last_tick_direction',
       'prev_price_24h', 'price_24h_pcnt', 'high_price_24h', 'low_price_24h',
       'prev_price_1h', 'mark_price', 'index_price', 'open_interest',
       'countdown_hour', 'turnover_24h', 'volume_24h', 'funding_rate',
       'predicted_funding_rate', 'next_funding_time',
       'predicted_delivery_price', 'total_turnover', 'total_volume',
       'delivery_fee_rate', 'delivery_time', 'price_1h_pcnt', 'open_value'],
      dtype='object')

In [6]:
session.orderbook(symbol='BTCUSD')['result'][1]

{'symbol': 'BTCUSD', 'price': '17006.5', 'side': 'Buy', 'size': 41}

In [5]:
#Stablishing Connection with the API (SPOT)
from pybit import spot
session_auth = spot.HTTP(
    endpoint='https://api.bybit.com',
    api_key = api_key_pw,
    api_secret= api_secret_pw
)

#Creating a Dataframe of BTC real price (1m)
frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="1m")["result"])
frame

,0,1,2,3,4,5,6,7,8,9,10
0,1670184180000,17085.09,17090,17083.54,17083.54,1.357338,1670184240000,23192.61825612,28,0,0
1,1670184240000,17083.54,17091.16,17083.54,17087.38,1.44123,1670184300000,24628.86500285,41,0,0
2,1670184300000,17087.38,17091,17084.86,17088.12,5.241285,1670184360000,89560.84206272,70,0,0
3,1670184360000,17088.12,17092.37,17088.12,17089.71,0.538057,1670184420000,9196.07978991,26,0,0
4,1670184420000,17089.71,17091.08,17082.16,17084,6.021156,1670184480000,102872.27492156,72,0,0
...,...,...,...,...,...,...,...,...,...,...,...
995,1670243880000,17266.2,17268.6,17260,17260,2.011893,1670243940000,34737.54427397,50,0,0
996,1670243940000,17260,17260,17238.05,17244.17,5.531612,1670244000000,95409.06276968,139,0,0
997,1670244000000,17244.17,17251.46,17232.86,17238.13,3.492526,1670244060000,60224.25838557,146,0,0
998,1670244060000,17238.13,17250,17233.03,17244.86,0.571212,1670244120000,9848.6621134,74,0,0


In [6]:
#This function gets Real BTC Price Data and creates a smooth dataframe that refreshes every 15 minutes
def get15minutedata():
    frame = pd.DataFrame(session_auth.query_kline(symbol="BTCUSDT", interval="15m")["result"])
    frame = frame.iloc[:,: 6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index, unit="ms")
    frame = frame.astype(float)
    return frame

In [7]:
df = get15minutedata()
df

,Open,High,Low,Close,Volume
Time,,,,,
2022-11-25 02:45:00,16511.04,16530.21,16508.40,16525.78,32.831399
2022-11-25 03:00:00,16525.78,16526.72,16497.72,16519.41,43.573818
2022-11-25 03:15:00,16519.41,16520.31,16471.45,16487.15,44.019146
2022-11-25 03:30:00,16487.15,16500.00,16483.23,16491.94,19.776946
2022-11-25 03:45:00,16491.94,16510.00,16484.61,16484.61,27.669884
...,...,...,...,...,...
2022-12-05 11:30:00,17288.88,17302.30,17285.56,17293.60,18.686844
2022-12-05 11:45:00,17293.60,17312.00,17290.00,17307.74,24.062538
2022-12-05 12:00:00,17307.74,17311.78,17233.79,17269.67,57.838734


In [11]:
#Function to apply some technical indicators from the ta library
def apply_technicals(df):
    df["K"] = ta.momentum.stochrsi(df.Close, window= 14)
    #df["D"] = df["K"].rolling(3).mean()
    df["RSI"] = ta.momentum.rsi(df.Close, window = 14)
    df.dropna(inplace=True)


In [12]:
apply_technicals(df)

In [13]:
df.head()

,Open,High,Low,Close,Volume,K,RSI
Time,,,,,,,
2022-11-25 15:45:00,16497.28,16516.74,16475.77,16511.44,60.156268,0.654032,52.592341
2022-11-25 16:00:00,16511.44,16513.99,16485.19,16486.10,41.123079,0.375105,47.086674
2022-11-25 16:15:00,16486.10,16499.31,16459.93,16481.95,67.988440,0.331858,46.233051
2022-11-25 16:30:00,16481.95,16492.73,16460.04,16485.76,58.910066,0.379822,47.179789
2022-11-25 16:45:00,16485.76,16496.32,16471.19,16485.67,33.161443,0.378752,47.158665


In [14]:
#Class function to create a trading system
class Signals:
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
    
    #Checking if we have a trigger in the last n time steps
    def get_trigger(self):
        df_2 = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df["RSI"].shift(i) < 20)
            df_2 = df_2.append(mask, ignore_index = True)
        return df_2.sum(axis= 0)
    
    # Is the trigger fulfilled and are all buying conditions fulfilled?
    def decide(self):
         self.df["trigger"] = np.where(self.get_trigger(), 1, 0)
         self.df["Buy"]= np.where((self.df.trigger) & (self.df.K < 0.20), 1, 0)


In [16]:
inst = Signals(df, 2)

inst.decide()

In [17]:
df[df.Buy==1]

,Open,High,Low,Close,Volume,K,RSI,trigger,Buy
Time,,,,,,,,,
2022-11-28 01:15:00,16388.00,16396.00,16192.95,16234.51,372.109590,0.000000,16.341903,1,1
2022-11-28 01:30:00,16234.51,16240.00,16114.32,16154.54,274.664868,0.000000,13.496908,1,1
2022-11-28 01:45:00,16154.54,16167.00,16099.47,16147.83,106.145603,0.000000,13.287875,1,1
2022-11-28 02:00:00,16147.83,16152.53,16056.02,16140.34,149.339512,0.000000,13.045008,1,1
2022-11-28 02:15:00,16140.34,16192.80,16103.35,16133.17,69.961093,0.000000,12.803755,1,1
2022-11-28 02:30:00,16133.17,16185.45,16119.47,16143.32,73.710590,0.090769,15.194770,1,1
2022-11-28 02:45:00,16143.32,16152.94,16106.03,16119.90,56.347533,0.053972,14.225473,1,1


In [ ]:
#Implementing the strategy with real time orders
#The function receives a quantity and waits for the buying conditions to be met
#We are going to buy when RSI < 20 and K < 0.2
#We are going to sell when one of these condition meets:
                    #Price falls by 2% (Loss)
                    #Price rises by 2% (Win)
                    #RSI > 45 

def strategy_long(qty, open_position = False):
    df= get15minutedata()
    apply_technicals(df)
    inst = Signals(df, 1)
    inst.decide()
    print(f'Current Close is '+str(df.Close.iloc[-1]))
    print(f'Current RSI is ' + str(df.RSI.iloc[-1]))
    print("-----------------------------------------")

    if df.Buy.iloc[-1]:
        from pybit import usdt_perpetual
        session = usdt_perpetual.HTTP(
        endpoint='https://api.bybit.com',
        api_key = api_key_pw,
        api_secret= api_secret_pw)

        buyprice = round(df.Close.iloc[-1],2)

        print("-----------------------------------------")

        print(f"Buyprice: {buyprice}")

        print("-----------------------------------------------------------------------------------------------------------------------------------------------")

        order = session.place_active_order(symbol="BTCUSDT",
                                                side="Buy",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=False,
                                                close_on_trigger=False,
                                                take_profit = round(buyprice * 1.02,2),
                                                stop_loss = round(buyprice * 0.98,2))
        print(order)

        btc_order_id = str(order['result']['order_id'])
        print("-----------------------------------------------------------------------------------------------------------------------------------------------")
        print(f"Order id: {btc_order_id}") 
        print("---------------------------------------------------")

        open_position = True
    while open_position:
        time.sleep(30)
        from pybit import spot
        session_auth = spot.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)
            
        df = get15minutedata()
        apply_technicals(df)
        print(f'Close: ' + str(df.Close.iloc[-1]))
        print(f'Target: ' + str(round(buyprice * 1.02, 2)) + "           Stop: " + str(round(buyprice * 0.98, 2)))
        print(f'RSI: ' + str(df.RSI.iloc[-1]))
        print("---------------------------------------------------")

        if df.Close[-1] > buyprice* 1.02 or df.Close[-1] < 0.98 * buyprice: 
            print("Closed Position")
            open_position = False
            break
        
        elif df.RSI[-1] > 45:
            session = usdt_perpetual.HTTP(
            endpoint='https://api.bybit.com',
            api_key = api_key_pw,
            api_secret= api_secret_pw)

            print(session.place_active_order(symbol="BTCUSDT",
                                                side="Sell",
                                                order_type="Market",
                                                qty= qty,
                                                time_in_force="GoodTillCancel",
                                                reduce_only=True,
                                                close_on_trigger=False))            
            
            print("Closed position")
            open_position = False
            break

In [22]:
#We activate our bot 
#Must have USDT deposited on the exchange for this to work (Futures)
#We define a quantity and refresh for new changes in price every 10 seconds
#Wait until the conditions are met for executing trades
while True: 
    strategy_long(0.001)
    time.sleep(10)

Current Close is 16952.87
Current Close is 16945.3
Current Close is 16941.63
Current Close is 16949.87
Current Close is 16944.7
Current Close is 16942.57
Current Close is 16948.39
Current Close is 16957.82
Current Close is 16955.51
Current Close is 16957.21
Current Close is 16955.79
Current Close is 16956.58
Current Close is 16944.0
Current Close is 16944.85
Current Close is 16943.84
Current Close is 16937.96
Current Close is 16934.34
Current Close is 16940.0
Current Close is 16953.0
Current Close is 16951.9
Current Close is 16952.19
Current Close is 16957.43
Current Close is 16958.56
Current Close is 16962.38
Current Close is 16970.0
Current Close is 16963.5
Current Close is 16963.47
Current Close is 16954.7
Current Close is 16953.36
Current Close is 16957.0
Current Close is 16959.27
Current Close is 16965.59
Current Close is 16963.59
Current Close is 16960.23
Current Close is 16965.0
Current Close is 16959.78
Current Close is 16964.48
Current Close is 16963.4
Current Close is 16950.0

KeyboardInterrupt: 